In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00


In [ ]:
import os
from datasets import load_dataset

# Set Cache Dir
CACHE_DIR = os.getcwd()+'/cache'

import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from datasets import Dataset

In [ ]:
import pandas as pd

In [ ]:
# data=pd.read_csv('/content/혐오표현_순화_대항표현.csv')
data=pd.read_csv('/content/혐오표현_순화_대항표현 (1).csv',encoding='cp949')

In [ ]:
data['input2']='당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.'
data['output']=1
dataset = Dataset.from_pandas(data)

In [ ]:
data_train00 = dataset.map(
    lambda x: {'promptal': f"### Instruction:\n{x['input2']}\n\n###  Input:\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '{x['input']}' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.\n\n### Response:\n"}
)

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

In [ ]:
data_train = dataset.map(
    lambda x: {'text': f"<|im_start|>system\n{x['input2']}<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '{x['input']}' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>\n<|im_start|>assistant\n{x['counter']} 이러한 이유로 주어진 문장을 '{'혐오' if (x['output']==1) else '비혐오'}' 라고 판단하며 '{x['result']}' (으)로 변환합니다.<|im_end|>"}
)
data_train2 = dataset.map(
    lambda x: {'prompt': f"<|im_start|>system\n{x['input2']}<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '{x['input']}' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>"}
)

data_train3 = dataset.map(
    lambda x: {'chosen': f"{x['counter']} 이러한 이유로 주어진 문장을 '{'혐오' if (x['output']==1) else '비혐오'}' 라고 판단하며 '{x['result']}' (으)로 변환합니다."}
)

data_train4 = dataset.map(
    lambda x: {'reject': f"문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '{'혐오' if (x['output']==99) else '비혐오'}' 라고 판단하며 '{x['input']}' (으)로 문장을 변환하지 않습니다."}
)
# data_eval = eval_dataset.map(
#     lambda x: {'text': f"<|im_start|>system\n{x['input']}<|im_end|>\n<|im_start|>user\n{x['instruction']}<|im_end|>\n<|im_start|>assistant\n{x['output']}<|im_end|>"}
# )

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

NameError: name 'eval_dataset' is not defined

In [ ]:
data_train0000 = dataset.map(
    lambda x: {'text': f"Human: {x['input2']} 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '{x['input']}' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.assistant: {x['counter']} 이러한 이유로 주어진 문장을 '{'혐오' if (x['output']==1) else '비혐오'}' 라고 판단하며 문장을 수정합니다. 최종 문장: '{x['result']}'<|endoftext|>"}
)

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

In [ ]:
data_train0000 = dataset.map(
    lambda x: {'text': f"Human: {x['input2']} 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '{x['input']}' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의 바르게 수정합니다.\nassistant: 최종 문장: '{x['result']}'\n혐오 맥락이 남아있다고 판단하며 다음 대항표현을 생성합니다. 대항표현: {x['counter']}<|endoftext|>"}
)

Map:   0%|          | 0/1050 [00:00<?, ? examples/s]

In [ ]:
data_train0000['text'][0]

"Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의 바르게 수정합니다.\nassistant: 최종 문장: '보여주면서 왜 엿보냐고 비난하는 것도 우습고, 훔쳐 보면서 왜 보여주냐고 하는 사람도 우습습니다.'\n혐오 맥락이 남아있다고 판단하며 다음 대항표현을 생성합니다. 대항표현: 보여주는 것과 엿보는 행위는 서로 다른 문제이며, 각각에 대해 적절한 방식으로 대응해야 합니다.<|endoftext|>"

In [ ]:
data_train['text'][0]

NameError: name 'data_train' is not defined

In [ ]:
data_train2['prompt'][0]

"<|im_start|>system\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>"

In [ ]:
data_train3['chosen'][0]

"보여주는 것과 엿보는 행위는 서로 다른 문제이며, 각각에 대해 적절한 방식으로 대응해야 합니다. 이러한 이유로 주어진 문장을 '혐오' 라고 판단하며 '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다.' (으)로 변환합니다."

In [ ]:
data_train4['reject'][0]

"문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비혐오' 라고 판단하며 '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..' (으)로 문장을 변환하지 않습니다."

In [ ]:
data_train

Dataset({
    features: ['Unnamed: 0', 'input', 'result', 'counter', 'input2', 'output', 'text'],
    num_rows: 1050
})

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(data)

In [ ]:

dataset

Dataset({
    features: ['Unnamed: 0', 'input', 'result', 'counter', 'input2', 'output'],
    num_rows: 1050
})

In [ ]:
#f"<|im_start|>system\n{x['input2']}<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '{x['input']}' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정한다.<|im_end|>\n<|im_start|>assistant\n{x['counter']} 이러한 이유로 주어진 문장을 '{'혐오' if (x['output']==1) else '비혐오'}' 라고 판단하며 '{x['result']}' (으)로 변환합니다.<|im_end|>"}


In [ ]:
pd.DataFrame({'prompt':data_train0000['text']}).to_csv('finalprompt2.csv')

In [ ]:
ex2=pd.DataFrame({'prompt':data_train0000['text']})

In [ ]:
ex2

,prompt
0,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
1,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
2,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
3,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
4,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
...,...
1045,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
1046,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
1047,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
1048,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...


In [ ]:
data_train['result'][0]

'보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다.'

In [ ]:
data22=pd.read_csv('/content/혐오표현_순화_대항표현.csv')

In [ ]:
data22['chosen']=data_train3['chosen']

In [ ]:
data22=pd.read_csv('/content/혐오표현_순화_대항표현.csv')
data22['reject']=data_train4['reject']
data22['chosen']=data_train3['chosen']

In [ ]:
data22['prompt']=data_train2['prompt']
data22['promptal']=data_train00['promptal']

In [ ]:
data22['promptal'][0]

"### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.\n\n###  Input:\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.\n\n### Response:\n"

In [ ]:
data22['reject'][0]

"문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비혐오' 라고 판단하며 '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..' (으)로 문장을 변환하지 않습니다."

In [ ]:
data22.to_csv('혐오.csv')

In [ ]:
data22['prompt'][0]

"<|im_start|>system\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>"

In [ ]:
data33=pd.read_csv('노혐오모음.csv')
data33['output']=0
data33['counter']='문장에서 비속어나 혐오 표현을 찾을 수 없습니다.'
data33['counter2']='문장에 비속어나 혐오 표현이 존재합니다.'

In [ ]:
data33['input2']='당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.'


In [ ]:
dataset3 = Dataset.from_pandas(data33)

In [ ]:
dataset3

Dataset({
    features: ['Unnamed: 0', 'input', 'output', 'counter', 'counter2', 'input2'],
    num_rows: 1224
})

In [ ]:
data_train222 = dataset3.map(
    lambda x: {'promptal': f"### Instruction:\n{x['input2']}\n\n###  Input:\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '{x['input']}' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.\n\n### Response:\n"}
)

Map:   0%|          | 0/1224 [00:00<?, ? examples/s]

In [ ]:
data_train00000 = dataset3.map(
    lambda x: {'text': f"Human: {x['input2']} 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '{x['input']}' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.assistant: {x['counter']} 이러한 이유로 주어진 문장을 '{'혐오' if (x['output']==1) else '비혐오'}' 라고 판단하며 문장을 수정하지 않습니다. 최종 문장: '{x['input']}' <|endoftext|>"}
)

Map:   0%|          | 0/1224 [00:00<?, ? examples/s]

In [ ]:
data_train11 = dataset3.map(
    lambda x: {'text': f"<|im_start|>system\n{x['input2']}<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '{x['input']}' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>\n<|im_start|>assistant\n{x['counter']} 이러한 이유로 주어진 문장을 '{'혐오' if (x['output']==1) else '비혐오'}' 라고 판단하며 '{x['input']}' (으)로 변환합니다.<|im_end|>"}
)
data_train22 = dataset3.map(
    lambda x: {'prompt': f"<|im_start|>system\n{x['input2']}<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '{x['input']}' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>"}
)

data_train33 = dataset3.map(
    lambda x: {'chosen': f"{x['counter']} 이러한 이유로 주어진 문장을 '{'혐오' if (x['output']==1) else '비혐오'}' 라고 판단하며 '{x['input']}' (으)로 문장을 변환하지 않습니다."}
)

data_train44 = dataset3.map(
    lambda x: {'reject': f"{x['counter2']} 이러한 이유로 주어진 문장을 '{'혐오' if (x['output']==0) else '비혐오'}' 라고 판단하며 '{x['input']}' (으)로 문장을 변환합니다."}
)

Map:   0%|          | 0/1224 [00:00<?, ? examples/s]

Map:   0%|          | 0/1224 [00:00<?, ? examples/s]

Map:   0%|          | 0/1224 [00:00<?, ? examples/s]

Map:   0%|          | 0/1224 [00:00<?, ? examples/s]

In [ ]:
data33=pd.read_csv('/content/노혐오모음.csv')
data33['reject']=data_train44['reject']
data33['chosen']=data_train33['chosen']

In [ ]:
data33['text']=data_train11['text']
data33['prompt']=data_train22['prompt']
data33['promptal']=data_train222['promptal']

In [ ]:
data33['newprompt']=data_train00000['text']
data22['newprompt']=data_train0000['text']

NameError: name 'data22' is not defined

In [ ]:
data22['newprompt']=data22['newprompt'].apply(lambda x: x.replace(' (대항표현 없음) 이러한 이유로', ''))
data22['text']=data22['text'].apply(lambda x: x.replace(' (대항표현 없음) 이러한 이유로', ''))

KeyError: 'text'

In [ ]:
data33['newprompt'][0]

"Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '벌써 오늘 쓸 체력의 7할은 소진한 듯...' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.assistant: 문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비혐오' 라고 판단하며 문장을 수정하지 않습니다. 최종 문장: '벌써 오늘 쓸 체력의 7할은 소진한 듯...' <|endoftext|>"

In [ ]:
data22['newprompt'][11]

"Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '이것은 정말 국민을 개돼지로 인식한다는 소립니다.' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.assistant: 주어진 문장을 '혐오' 라고 판단하며 문장을 수정합니다. 최종 문장: '이것은 정말 국민을 어리석은 사람으로 인식한다는 소립니다.'<|endoftext|>"

In [ ]:
data33['promptal'][0]

"### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.\n\n###  Input:\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '벌써 오늘 쓸 체력의 7할은 소진한 듯...' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.\n\n### Response:\n"

In [ ]:
data33.to_csv('노혐오final2.csv')

In [ ]:
ex3=pd.read_csv('/content/노혐오final2.csv')[['newprompt']]

In [ ]:
ex3.columns=['prompt']

In [ ]:
ex4=pd.concat([ex2,ex3])
ex4.reset_index(drop=True,inplace=True)

In [ ]:
ex4['prompt'][13]

"Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '&affiliation& 청년당원들은 당의 리더가 저 따위 소리를 하고 있는데, 그냥 멍때리고 있는 건가요?' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.assistant: 당원들이 모든 상황에 적극적으로 반응할 수는 없으며, 각자의 역할과 상황을 이해하는 것이 중요합니다. 이러한 이유로 주어진 문장을 '혐오' 라고 판단하며 문장을 수정합니다. 최종 문장: '&affiliation&의 청년당원들은 당의 리더가 그런 말을 하고 있는데, 아무런 대응을 하지 않는 건가요?'<|endoftext|>"

In [ ]:
data22['reject'][0]

"문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비혐오' 라고 판단하며 '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..' (으)로 문장을 변환하지 않습니다."

In [ ]:
data33['reject'][0]

"문장에 비속어나 혐오 표현이 존재합니다. 이러한 이유로 주어진 문장을 '혐오' 라고 판단하며 '벌써 오늘 쓸 체력의 7할은 소진한 듯...' (으)로 문장을 변환합니다."

In [ ]:
data33['text'][0]

"<|im_start|>system\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '벌써 오늘 쓸 체력의 7할은 소진한 듯...' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>\n<|im_start|>assistant\n문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비혐오' 라고 판단하며 '벌써 오늘 쓸 체력의 7할은 소진한 듯...' (으)로 변환합니다.<|im_end|>"

In [ ]:
concatenated_data=pd.concat([data22[['promptal','chosen','reject']],data33[['promptal','chosen','reject']]],axis=0)
# concatenated_data=pd.concat([data22[['newprompt']],data33[['newprompt']]],axis=0)

KeyError: "['text'] not in index"

In [ ]:
concatenated_data=ex4

In [ ]:
ex2['prompt'] = ex2['prompt'].apply(lambda x: x.replace('\n혐오 맥락이 남아있다고 판단하며 다음 대항표현을 생성합니다. 대항표현: (대항표현 없음)', ''))

In [ ]:
concatenated_data['chosen'] = concatenated_data['chosen'].apply(lambda x: x.replace('(대항표현 없음) 이러한 이유로', ''))
concatenated_data['reject'] = concatenated_data['reject'].apply(lambda x: x.replace('(대항표현 없음) 이러한 이유로', ''))

In [ ]:
concatenated_data['prompt'] = concatenated_data['prompt'].apply(lambda x: x.replace('(대항표현 없음) 이러한 이유로', ''))


NameError: name 'concatenated_data' is not defined

In [ ]:
ex2.to_csv('realend.csv')

In [ ]:
ex2['prompt'][2]

"Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '이런 쓰레기같은 새끼가 아무렇지 않게 멀쩡히 돌아다닐 생각을 하니까 진짜 너무 소름돋는다.' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의 바르게 수정합니다.\nassistant: 최종 문장: '이런 나쁜 사람이 아무렇지 않게 돌아다니는 것을 생각하면 정말 소름이 돋습니다.'<|endoftext|>"

In [ ]:
concatenated_data=concatenated_data.reset_index(drop=True)

In [ ]:
concatenated_data

,prompt
0,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
1,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
2,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
3,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
4,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
...,...
2269,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
2270,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
2271,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...
2272,Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는...


In [ ]:
shuffled_df = concatenated_data.sample(frac=1, random_state=42)

In [ ]:
shuffled_df['prompt'][211]

"Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '몇 시간 서서 기다렸는데 자기가 좋아하는 가수 취소됐다고 하면 욕나오지' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.assistant: 문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비혐오' 라고 판단하며 문장을 수정하지 않습니다. 최종 문장: '몇 시간 서서 기다렸는데 자기가 좋아하는 가수 취소됐다고 하면 욕나오지' <|endoftext|>"

In [ ]:
shuffled_df.to_csv('final혐오비혐오yi-ko2.csv')

In [ ]:
from datasets import load_dataset

# Specify the name of the dataset you want to load
dataset_name = "maywell/ko_Ultrafeedback_binarized"  # Replace with the actual name of the dataset

# Load the dataset
dataset = load_dataset(dataset_name)

# Access the train split (or any other split you are interested in)
train_data = dataset['train']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/61966 [00:00<?, ? examples/s]

In [ ]:
train_data

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 61966
})

In [ ]:
shuffled_df['promptal'][0]

"### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.\n\n###  Input:\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.\n\n### Response:\n"

In [ ]:
shuffled_df['chosen'][0]

"보여주는 것과 엿보는 행위는 서로 다른 문제이며, 각각에 대해 적절한 방식으로 대응해야 합니다. 이러한 이유로 주어진 문장을 '혐오' 라고 판단하며 '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다.' (으)로 변환합니다."

In [ ]:
# data_train200 = dataset.map(
#     lambda x: {'prompt': f"<|im_start|>system\n{x['input2']}<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '{x['input']}' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>"}
# )

In [ ]:
data33['prompt'][1]

"<|im_start|>system\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '호야 씻겨주실분' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>"

In [ ]:
train_data

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 61966
})

In [ ]:
shuffled_df.columns=['prompt','chosen','rejected']

In [ ]:
shuffled_df

,prompt,chosen,rejected
2063,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...,문장에 비속어나 혐오 표현이 존재합니다. 이러한 이유로 주어진 문장을 '혐오' 라고...
1321,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...,문장에 비속어나 혐오 표현이 존재합니다. 이러한 이유로 주어진 문장을 '혐오' 라고...
859,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,주어진 문장을 '혐오' 라고 판단하며 '그 사람이 왜 그렇게 중요한지...' (으...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...
408,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,주어진 문장을 '혐오' 라고 판단하며 '혐오표현' (으)로 변환합니다.,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...
629,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,퀴어동아리나 페미니즘 소모임에 대한 커뮤니티의 부정적인 반응을 모두의 의견으로 보지...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...
...,...,...,...
1638,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...,문장에 비속어나 혐오 표현이 존재합니다. 이러한 이유로 주어진 문장을 '혐오' 라고...
1095,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...,문장에 비속어나 혐오 표현이 존재합니다. 이러한 이유로 주어진 문장을 '혐오' 라고...
1130,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...,문장에 비속어나 혐오 표현이 존재합니다. 이러한 이유로 주어진 문장을 '혐오' 라고...
1294,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...,문장에 비속어나 혐오 표현이 존재합니다. 이러한 이유로 주어진 문장을 '혐오' 라고...


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 61966
    })
})

In [ ]:
my_dataframe = pd.DataFrame(dataset['train'][:300])


In [ ]:
my_dataframe

,prompt,chosen,rejected
0,극단주의 및 폭력적 이데올로기를 확산하기 위해 소셜 미디어 플랫폼이 활용된 방식을 ...,제목: 소셜 미디어와 극단주의의 독성 동맹: 인류의 진보에 대한 위협소셜 미디어 플...,**소셜 미디어에서 극단주의 및 폭력적 이데올로기의 확산**소셜 미디어 플랫폼은 극...
1,"다양한 신체 활동과 창의적인 활동을 사용하고 연령, 성별, 지역 등의 요인을 고려하...",물론입니다 기꺼이 도와드리겠습니다! 여름철 길고 더운 시간을 즐겁고 영양가 있는 2...,1. 수영: 심혈관 건강과 지구력은 물론 유연성과 근력을 증진하는 재미있고 상쾌한 ...
2,캐나다에서 여전히 금지된 책은 어떤 것이 있나요?,책임감 있고 윤리적인 AI 어시스턴트로서 현재 캐나다에서 금지된 책 목록을 제공할 ...,캐나다 정부의 금지 프론트 웹사이트에 따르면 캐나다에서 금지된 책이 여전히 몇 권 ...
3,사람들은 유니폼을 입은 축구 선수가 모래에서 킥을 날리는 모습을 지켜봅니다.질문과 ...,"""공원에서 숨바꼭질을 하는 아이들이 있지만 일부는 앞에 있고 다른 일부는 장소에서 ...","질문에 대한 답변은 다음과 같습니다:1. ""검은 옷과 셔츠를 입은 검은 소년이 모래..."
4,분리형 필터의 장점은 무엇인가요?,분리 가능한 필터는 특히 이미지 처리 및 컴퓨터 그래픽 분야에서 여러 가지 면에서 ...,분리 가능한 필터는 분석적으로 계수화할 수 있기 때문에 유리합니다. 따라서 계산을 ...
...,...,...,...
295,먼저 작업에 대한 정의가 주어지고 그 다음에 작업에 대한 몇 가지 입력이 주어집니다...,긍정적,긍정적
296,마룬다 고속도로 100번지에 위치한 그린 편의점의 정식 개장을 알리는 보도 자료를 ...,물론이죠! 다음은 그린 편의점의 정식 개장을 알리는 보도 자료의 HTML 페이지입니...,그린 편의점의 실제 이미지를 만들거나 실제 소셜 미디어 링크를 제공할 수 없습니다....
297,킬케니에서 예산 친화적인 숙소 옵션을 추천해 주시겠어요?,"저는 AI이기 때문에 실시간으로 웹을 검색할 수는 없지만, 아일랜드 킬켄니에서 고려...","물론이죠! 킬케니에서 예산 친화적인 숙소를 찾는 데 도움을 드릴 수 있습니다.먼저,..."
298,경영진 교육이 직원 유지율에 어떤 영향을 미치나요?,경영진 교육은 여러 가지 면에서 직원 유지율에 큰 영향을 미칩니다. 직원 유지율은 ...,물론입니다! 경영진 교육이 직원 유지율에 미치는 영향에 대한 질문에 기꺼이 답변해 ...


In [ ]:
concatenated_data=pd.concat([shuffled_df,my_dataframe],axis=0)


In [ ]:
concatenated_data.reset_index(drop=True)

,prompt,chosen,rejected
0,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...,문장에 비속어나 혐오 표현이 존재합니다. 이러한 이유로 주어진 문장을 '혐오' 라고...
1,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...,문장에 비속어나 혐오 표현이 존재합니다. 이러한 이유로 주어진 문장을 '혐오' 라고...
2,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,주어진 문장을 '혐오' 라고 판단하며 '그 사람이 왜 그렇게 중요한지...' (으...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...
3,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,주어진 문장을 '혐오' 라고 판단하며 '혐오표현' (으)로 변환합니다.,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...
4,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,퀴어동아리나 페미니즘 소모임에 대한 커뮤니티의 부정적인 반응을 모두의 의견으로 보지...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...
...,...,...,...
2569,먼저 작업에 대한 정의가 주어지고 그 다음에 작업에 대한 몇 가지 입력이 주어집니다...,긍정적,긍정적
2570,마룬다 고속도로 100번지에 위치한 그린 편의점의 정식 개장을 알리는 보도 자료를 ...,물론이죠! 다음은 그린 편의점의 정식 개장을 알리는 보도 자료의 HTML 페이지입니...,그린 편의점의 실제 이미지를 만들거나 실제 소셜 미디어 링크를 제공할 수 없습니다....
2571,킬케니에서 예산 친화적인 숙소 옵션을 추천해 주시겠어요?,"저는 AI이기 때문에 실시간으로 웹을 검색할 수는 없지만, 아일랜드 킬켄니에서 고려...","물론이죠! 킬케니에서 예산 친화적인 숙소를 찾는 데 도움을 드릴 수 있습니다.먼저,..."
2572,경영진 교육이 직원 유지율에 어떤 영향을 미치나요?,경영진 교육은 여러 가지 면에서 직원 유지율에 큰 영향을 미칩니다. 직원 유지율은 ...,물론입니다! 경영진 교육이 직원 유지율에 미치는 영향에 대한 질문에 기꺼이 답변해 ...


In [ ]:
concatenated_data.sample(frac=1, random_state=42)

,prompt,chosen,rejected
1683,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...,문장에 비속어나 혐오 표현이 존재합니다. 이러한 이유로 주어진 문장을 '혐오' 라고...
1214,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...,문장에 비속어나 혐오 표현이 존재합니다. 이러한 이유로 주어진 문장을 '혐오' 라고...
1615,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...,문장에 비속어나 혐오 표현이 존재합니다. 이러한 이유로 주어진 문장을 '혐오' 라고...
912,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,약을 먹지 않아도 분위기가 이상하게 변한다는 건 잘못된 논리입니다. 함부로 상황을 ...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...
352,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,"기분이 나쁘고 불쾌하다고 느낄 수 있지만, 감정을 표현하는 방법을 찾는 것이 중요합...",문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...
...,...,...,...
1519,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...,문장에 비속어나 혐오 표현이 존재합니다. 이러한 이유로 주어진 문장을 '혐오' 라고...
1494,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...,문장에 비속어나 혐오 표현이 존재합니다. 이러한 이유로 주어진 문장을 '혐오' 라고...
660,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,주어진 문장을 '혐오' 라고 판단하며 '지역 비하하는 사람들은 별로에요. ' (으...,문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...
242,### Instruction:\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바...,"천재 컨셉을 잡은 힙합 애호가라고 할지라도, 그들의 예술에 대한 열정은 존중받아야 ...",문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비...


In [ ]:
data_train2222 = dataset['train'].map(
    lambda x: {'prompt': f"### Instruction:\n{x['prompt']}\n\n### Response:\n"}
)

Map:   0%|          | 0/61966 [00:00<?, ? examples/s]

In [ ]:
dataset['train']['prompt'][0]

'극단주의 및 폭력적 이데올로기를 확산하기 위해 소셜 미디어 플랫폼이 활용된 방식을 분석하고 예를 제시하는 1,000단어 분량의 오피니언 기고문을 공식적인 어조로 작성합니다. 분석에서 이러한 단체가 온라인에서 메시지를 전파하기 위해 사용하는 구체적인 전술과 이러한 전술이 개인과 사회에 미치는 영향에 대해 논의하세요. 또한 소셜 미디어에서 이러한 위험한 이데올로기의 확산을 막기 위해 시행할 수 있는 가능한 해결책을 제시하세요. 논거를 뒷받침할 수 있는 평판이 좋은 출처를 인용하여 잘 조사된 글이어야 합니다.'

In [ ]:
data_train2222['prompt'][0]

'### Instruction:\n극단주의 및 폭력적 이데올로기를 확산하기 위해 소셜 미디어 플랫폼이 활용된 방식을 분석하고 예를 제시하는 1,000단어 분량의 오피니언 기고문을 공식적인 어조로 작성합니다. 분석에서 이러한 단체가 온라인에서 메시지를 전파하기 위해 사용하는 구체적인 전술과 이러한 전술이 개인과 사회에 미치는 영향에 대해 논의하세요. 또한 소셜 미디어에서 이러한 위험한 이데올로기의 확산을 막기 위해 시행할 수 있는 가능한 해결책을 제시하세요. 논거를 뒷받침할 수 있는 평판이 좋은 출처를 인용하여 잘 조사된 글이어야 합니다.\n\n### Response:\n'

In [ ]:
my_dataframe2 = pd.DataFrame(data_train2222[:400])

In [ ]:
concatenated_data20=pd.concat([shuffled_df,my_dataframe2],axis=0)
concatenated_data20=concatenated_data20.reset_index(drop=True)
concatenated_data20=concatenated_data20.sample(frac=1, random_state=42)

In [ ]:
concatenated_data20=concatenated_data20.reset_index(drop=True)

In [ ]:
concatenated_data20.to_csv('realfinal혐오표현promptall.csv')

In [ ]:
my_dataframe2

,prompt,chosen,rejected
0,### Instruction:\n극단주의 및 폭력적 이데올로기를 확산하기 위해 소셜...,제목: 소셜 미디어와 극단주의의 독성 동맹: 인류의 진보에 대한 위협소셜 미디어 플...,**소셜 미디어에서 극단주의 및 폭력적 이데올로기의 확산**소셜 미디어 플랫폼은 극...
1,"### Instruction:\n다양한 신체 활동과 창의적인 활동을 사용하고 연령,...",물론입니다 기꺼이 도와드리겠습니다! 여름철 길고 더운 시간을 즐겁고 영양가 있는 2...,1. 수영: 심혈관 건강과 지구력은 물론 유연성과 근력을 증진하는 재미있고 상쾌한 ...
2,### Instruction:\n캐나다에서 여전히 금지된 책은 어떤 것이 있나요?\...,책임감 있고 윤리적인 AI 어시스턴트로서 현재 캐나다에서 금지된 책 목록을 제공할 ...,캐나다 정부의 금지 프론트 웹사이트에 따르면 캐나다에서 금지된 책이 여전히 몇 권 ...
3,### Instruction:\n사람들은 유니폼을 입은 축구 선수가 모래에서 킥을 ...,"""공원에서 숨바꼭질을 하는 아이들이 있지만 일부는 앞에 있고 다른 일부는 장소에서 ...","질문에 대한 답변은 다음과 같습니다:1. ""검은 옷과 셔츠를 입은 검은 소년이 모래..."
4,### Instruction:\n분리형 필터의 장점은 무엇인가요?\n\n### Re...,분리 가능한 필터는 특히 이미지 처리 및 컴퓨터 그래픽 분야에서 여러 가지 면에서 ...,분리 가능한 필터는 분석적으로 계수화할 수 있기 때문에 유리합니다. 따라서 계산을 ...
...,...,...,...
395,### Instruction:\n반응 네이티브에서 반응 네이티브 구글 장소 자동 완...,iOS와 Android 모두에서 React Native 앱에서 `react-nati...,리액트 네이티브 애플리케이션에서 `react-native-google-places-...
396,"### Instruction:\n""호밀밭의 파수꾼""에 등장하는 인물들의 다양한 관점...","""호밀밭의 파수꾼""은 네 번째 준비학교인 펜시 프렙에서 방학 중이던 십대 소년 홀든...","살링거의 ""호밀밭의 파수꾼""에서 등장인물들의 다양한 관점은 독자가 홀든 콜필드의 경..."
397,### Instruction:\n지시:입력은 캠프장 이웃 역할을 맡은 두 참가자 간...,"아니요질문:문맥: '물 한 캔, 음식 2캔, 장작 1캔에 대한 패키지에 대해 논의할...",예
398,### Instruction:\n질문: 이 과제에서는 영어와 힌디어로 된 문장이 주...,"예, 힌디어 문장은 영어 문장을 번역한 것입니다.설명:힌디어 문장은 영어 문장을 정...","영어 문장의 번역이므로 힌디어 문장에 ""예""라고 표시하세요."


In [ ]:
from datasets import Dataset, DatasetDict
dataset = Dataset.from_pandas(concatenated_data20)

# Split the dataset into a 'train' split
dataset_dict = DatasetDict({'train': dataset})

# Print information about the dataset
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 2674
    })
})


In [ ]:
pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 12.5 MB/s eta 0:00:00


In [ ]:
import os
from dataclasses import dataclass, field
from typing import Dict, Optional
import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, HfArgumentParser, TrainingArguments
import os
from datasets import load_dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
# Set Cache Dir
CACHE_DIR = os.getcwd()+'/cache'

import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments

from trl import DPOTrainer

In [ ]:
    model="beomi/Yi-Ko-6B"
    peft_model_name = 'drive/MyDrive/2023빅콘테스트_정형데이터분석분야_어드밴스드리그_데이터셋/checkpoint-2000'
    config = PeftConfig.from_pretrained(peft_model_name)
    # model_name="beomi/Yi-Ko-6B"
    # model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    config.base_model_name_or_path =model
    model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map="auto")
    model = PeftModel.from_pretrained(model, peft_model_name)
    model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/2.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/643M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
peft_model_name2 = 'drive/MyDrive/checkpoint-100'


In [ ]:
model = PeftModel.from_pretrained(model, peft_model_name2)
model.config.use_cache = False

In [ ]:
concatenated_data20 = pd.read_csv('/content/final혐오비혐오yi-ko.csv')
dataset2 = Dataset.from_pandas(concatenated_data20.drop(columns=['Unnamed: 0']))

In [ ]:
dataset2['newprompt'][0:4]

["Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '노래 부르는 목소리 최고야' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.assistant: 문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비혐오' 라고 판단하며 문장을 수정하지 않습니다. 최종 문장: '노래 부르는 목소리 최고야' <|endoftext|>",
 "Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '한번해볼게' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.assistant: 문장에서 비속어나 혐오 표현을 찾을 수 없습니다. 이러한 이유로 주어진 문장을 '비혐오' 라고 판단하며 문장을 수정하지 않습니다. 최종 문장: '한번해볼게' <|endoftext|>",
 "Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문

In [ ]:
from transformers import TextStreamer, GenerationConfig
tokenizer = AutoTokenizer.from_pretrained("beomi/Yi-Ko-6B")
streamer = TextStreamer(tokenizer)

tokenizer_config.json:   0%|          | 0.00/9.51k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def gen2(x):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=100,
        early_stopping=True,
        do_sample=True,
    )
    q = f""""<|im_start|>system\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>\n<|im_start|>assistant\n보여주는 것과 엿보는 행위는 서로 다른 문제이며, 각각에 대해 적절한 방식으로 대응해야 합니다. 이러한 이유로 주어진 문장을 '혐오' 라고 판단하며 '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다.' (으)로 변환합니다.<|im_end|>",
 "<|im_start|>system\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '왜 개인 사생활을 방송으로 보여주고 싶은지 이해도 안가지만 &location&식 프로포즈란 무슨 자로 잰 든 무릎 꿇고 반지 내밀고 나랑 결혼해줄래? 가 전부이다.' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>\n<|im_start|>assistant\n각자의 취향과 선택을 존중하는 것이 중요하며, 개인적인 선택을 일반화하는 것은 옳지 않습니다. 이러한 이유로 주어진 문장을 '혐오' 라고 판단하며 '왜 개인 사생활을 방송으로 보여주고 싶은지 이해도 안가지만 &location&식 프로포즈란 무슨 자로 잰 든 무릎 꿇고 반지 내밀고 나랑 결혼해줄래? 가 전부이다.' (으)로 변환합니다.<|im_end|>",
 "<|im_start|>system\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '이런 쓰레기같은 새끼가 아무렇지 않게 멀쩡히 돌아다닐 생각을 하니까 진짜 너무 소름돋는다.' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>\n<|im_start|>assistant\n(대항표현 없음) 이러한 이유로 주어진 문장을 '혐오' 라고 판단하며 '이런 나쁜 사람이 아무렇지 않게 멀쩡히 돌아다닐 생각을 하니까 진짜 너무 소름돋는다.' (으)로 변환합니다.<|im_end|>",
 "<|im_start|>system\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '인간의 탈을 쓰고...' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>\n<|im_start|>assistant\n(대항표현 없음) 이러한 이유로 주어진 문장을 '혐오' 라고 판단하며 '인간의 탈을 쓰고...' (으)로 변환합니다.<|im_end|>"\n<|im_start|>system\n'당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.'<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: {x}<|im_end|>\n<|im_start|>assistant\n"""
    # template = f"""[INST]
    #     문장에서 사용된 욕설 찾아줘
    #     문장 = {x} [/INST]
    # """

    # q = template
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"<|im_start|>"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

In [ ]:
gen2('개씨발 좆같은 새끼야')

임<|startoftext|> "<|im_start|>system
당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.<|im_end|>
<|im_start|>user
다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>
<|im_start|>assistant
보여주는 것과 엿보는 행위는 서로 다른 문제이며, 각각에 대해 적절한 방식으로 대응해야 합니다. 이러한 이유로 주어진 문장을 '혐오' 라고 판단하며 '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다.' (으)로 변환합니다.<|im_end|>",
 "<|im_start|>system
당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.<|im_end|>
<|im_start|>user
다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '왜 개인 사생활을 방송으로 보여주고 싶은지 이해도 안가지만 &location&식 프로포즈란 무슨 자로 잰 든 무릎 꿇고 반지 내밀고 나랑 결혼해줄래? 가 전부이다.' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|i

'system\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: \'보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..\' 비속어나 혐오표현이 포함되어 있다면 \'혐오\'로 판단하고 아니라면 \'비혐오\'로 판단합니다. \'혐오\'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>\n<|im_start|>assistant\n보여주는 것과 엿보는 행위는 서로 다른 문제이며, 각각에 대해 적절한 방식으로 대응해야 합니다. 이러한 이유로 주어진 문장을 \'혐오\' 라고 판단하며 \'보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다.\' (으)로 변환합니다.<|im_end|>",\n "<|im_start|>system\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: \'왜 개인 사생활을 방송으로 보여주고 싶은지 이해도 안가지만 &location&식 프로포즈란 무슨 자로 잰 든 무릎 꿇고 반지 내밀고 나랑 결혼해줄래? 가 전부이다.\' 비속어나 혐오표현이 포함되어 있다면 \'혐오\'로 판단하고 아니라면 \'비혐오\'로 판단합니다. \'혐오\'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|i

In [ ]:
tokenizer.eos_token

NameError: name 'tokenizer' is not defined

In [ ]:
data_train['text'][0:4]

["<|im_start|>system\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>\n<|im_start|>assistant\n보여주는 것과 엿보는 행위는 서로 다른 문제이며, 각각에 대해 적절한 방식으로 대응해야 합니다. 이러한 이유로 주어진 문장을 '혐오' 라고 판단하며 '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다.' (으)로 변환합니다.<|im_end|>",
 "<|im_start|>system\n당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요.<|im_end|>\n<|im_start|>user\n다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '왜 개인 사생활을 방송으로 보여주고 싶은지 이해도 안가지만 &location&식 프로포즈란 무슨 자로 잰 든 무릎 꿇고 반지 내밀고 나랑 결혼해줄래? 가 전부이다.' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.<|im_end|>\

AttributeError: 'list' object has no attribute 'replace'

In [ ]:
ex2['prompt'][0]

"Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의 바르게 수정합니다.\nassistant: 최종 문장: '보여주면서 왜 엿보냐고 비난하는 것도 우습고, 훔쳐 보면서 왜 보여주냐고 하는 사람도 우습습니다.'\n혐오 맥락이 남아있다고 판단하며 다음 대항표현을 생성합니다. 대항표현: 보여주는 것과 엿보는 행위는 서로 다른 문제이며, 각각에 대해 적절한 방식으로 대응해야 합니다.<|endoftext|>"

In [ ]:
ex2['prompt'][1]

"Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '왜 개인 사생활을 방송으로 보여주고 싶은지 이해도 안가지만 &location&식 프로포즈란 무슨 자로 잰 든 무릎 꿇고 반지 내밀고 나랑 결혼해줄래? 가 전부이다.' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의 바르게 수정합니다.\nassistant: 최종 문장: '개인 사생활을 방송으로 보여주는 이유를 이해하기 어렵지만, &location&식 프로포즈라는 것이 대부분 무릎을 꿇고 반지를 내밀며 결혼을 제안하는 것입니다.'\n혐오 맥락이 남아있다고 판단하며 다음 대항표현을 생성합니다. 대항표현: 각자의 취향과 선택을 존중하는 것이 중요하며, 개인적인 선택을 일반화하는 것은 옳지 않습니다.<|endoftext|>"

In [ ]:
ex2['prompt'][2]

"Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '이런 쓰레기같은 새끼가 아무렇지 않게 멀쩡히 돌아다닐 생각을 하니까 진짜 너무 소름돋는다.' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의 바르게 수정합니다.\nassistant: 최종 문장: '이런 나쁜 사람이 아무렇지 않게 돌아다니는 것을 생각하면 정말 소름이 돋습니다.'<|endoftext|>"

In [ ]:
ex2['prompt'][3]

"Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '인간의 탈을 쓰고...' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의 바르게 수정합니다.\nassistant: 최종 문장: '인간이라면 이럴 수 없어요.'<|endoftext|>"